# Data Job Postings Analysis (Hugging Face Dataset)

This project analyzes job postings for data-related roles (e.g., Data Analyst, Data Scientist etc.) using the Hugging Face `data_jobs` dataset.  
We explore role distribution, skill demand, salary trends, and job locations to uncover hiring patterns in the data field.


### 1. Import Libraries & Load Data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import ast

print('Libraries loaded - ready to explore data jobs!')

Libraries loaded - ready to explore data jobs!


In [2]:
from datasets import load_dataset
from pathlib import Path

# Create data_raw folder
raw_path = Path("../data_raw")
raw_path.mkdir(parents=True, exist_ok=True)

# Load dataset from Hugging Face
dataset = load_dataset("lukebarousse/data_jobs")
df = dataset["train"].to_pandas()

# Save raw snapshot to data_raw
df.to_parquet(raw_path / "data_jobs_raw.parquet", index=False)

print(" Raw dataset loaded and saved to: data_raw/data_jobs_raw.parquet")


 Raw dataset loaded and saved to: data_raw/data_jobs_raw.parquet


In [3]:
# Reload the dataset from saved Parquet file

df = pd.read_parquet(raw_path / "data_jobs_raw.parquet")

# Preview shape
print(f'Dataset loaded : {df.shape}')
df.head(3)


Dataset loaded : (785741, 17)


,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,salary_year_avg,salary_hour_avg,company_name,job_skills,job_type_skills
0,Senior Data Engineer,Senior Clinical Data Engineer / Principal Clin...,"Watertown, CT",via Work Nearby,Full-time,False,"Texas, United States",2023-06-16 13:44:15,False,False,United States,None,NaN,NaN,Boehringer Ingelheim,None,None
1,Data Analyst,Data Analyst,"Guadalajara, Jalisco, Mexico",via BeBee México,Full-time,False,Mexico,2023-01-14 13:18:07,False,False,Mexico,None,NaN,NaN,Hewlett Packard Enterprise,"['r', 'python', 'sql', 'nosql', 'power bi', 't...","{'analyst_tools': ['power bi', 'tableau'], 'pr..."
2,Data Engineer,"Data Engineer/Scientist/Analyst, Mid or Senior...","Berlin, Germany",via LinkedIn,Full-time,False,Germany,2023-10-10 13:14:55,False,False,Germany,None,NaN,NaN,ALPHA Augmented Services,"['python', 'sql', 'c#', 'azure', 'airflow', 'd...","{'analyst_tools': ['dax'], 'cloud': ['azure'],..."


### 2. Inspect the data

2.1 Inspect the structure and null values in each column

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 785741 entries, 0 to 785740
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   job_title_short        785741 non-null  object 
 1   job_title              785740 non-null  object 
 2   job_location           784696 non-null  object 
 3   job_via                785733 non-null  object 
 4   job_schedule_type      773074 non-null  object 
 5   job_work_from_home     785741 non-null  bool   
 6   search_location        785741 non-null  object 
 7   job_posted_date        785741 non-null  object 
 8   job_no_degree_mention  785741 non-null  bool   
 9   job_health_insurance   785741 non-null  bool   
 10  job_country            785692 non-null  object 
 11  salary_rate            33067 non-null   object 
 12  salary_year_avg        22003 non-null   float64
 13  salary_hour_avg        10662 non-null   float64
 14  company_name           785723 non-nu

2.2 Missing values by column

In [5]:
df.isna().sum().sort_values(ascending = False)

salary_hour_avg          775079
salary_year_avg          763738
salary_rate              752674
job_type_skills          117037
job_skills               117037
job_schedule_type         12667
job_location               1045
job_country                  49
company_name                 18
job_via                       8
job_title                     1
job_title_short               0
job_work_from_home            0
job_no_degree_mention         0
job_posted_date               0
search_location               0
job_health_insurance          0
dtype: int64

2.3 Descriptive statistics (Numerical columns)

In [6]:
df.describe()

,salary_year_avg,salary_hour_avg
count,22003.000000,10662.000000
mean,123286.274072,47.016598
std,48312.449482,21.890738
min,15000.000000,8.000000
25%,90000.000000,27.500000
50%,115000.000000,45.980000
75%,150000.000000,61.159996
max,960000.000000,391.000000


2.4 Check for duplicates

In [7]:
df.duplicated().sum()

np.int64(101)

2.5 Unique value counts for key columns

In [8]:
for col in ['job_title_short', 'job_country', 'company_name', 'job_schedule_type']:
    print(f"{col} unique values:", df[col].nunique())


job_title_short unique values: 10
job_country unique values: 160
company_name unique values: 139982
job_schedule_type unique values: 47


2.6 Data range check

In [9]:
df['job_posted_date'].min(), df['job_posted_date'].max()

('2023-01-01 00:00:04', '2023-12-31 23:59:58')

### 3. Clean Data

In [10]:
# Strip whitespace and standardize column names
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Clean job_skills_column
df['job_skills'] = df['job_skills'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Convert job_posted_date to datetime format
df['job_posted_date'] = pd.to_datetime(df['job_posted_date'], errors = 'coerce')

# Drop mostly empty columns
df.drop(columns=['salary_hour_avg', 'salary_rate'], inplace = True)

# Drop rows where job_title is missing
df = df.dropna(subset =['job_title'])

#  Fill missing values
df.loc[:,'job_schedule_type'] = df['job_schedule_type'].fillna('Unknown')
df.loc[:,'job_location'] = df['job_location'].fillna('Unknown')
df.loc[:,'job_country'] = df['job_country'].fillna('Unknown')
df.loc[:,'company_name'] = df['company_name'].fillna('Unknown')
df.loc[:,'job_via'] = df['job_via'].fillna('Unknown')

### 4 Final Check

In [11]:
df.sample(3)

,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_year_avg,company_name,job_skills,job_type_skills
612876,Data Analyst,Senior BI Engineer,Poland,via Trabajo.org,Full-time,False,Poland,2023-02-14 12:37:57,True,False,Poland,NaN,Luxoft Poland Sp. z o.o.,"[python, databricks, react, microstrategy, tab...","{'analyst_tools': ['microstrategy', 'tableau']..."
327015,Data Engineer,Sr data engineer,"Corona, CA",via Talent.com,Full-time,False,Sudan,2023-06-26 00:14:26,False,False,Sudan,NaN,Supernal,"[sql, azure, snowflake, bigquery, aws, redshif...","{'cloud': ['azure', 'snowflake', 'bigquery', '..."
149109,Data Analyst,Data Analyst,Australia,via BeBee Australia,Full-time,False,Australia,2023-12-01 08:19:11,True,False,Australia,NaN,Clicks IT Recruitment,"[sql, go, sql server, power bi]","{'analyst_tools': ['power bi'], 'databases': [..."


### 5. Save Cleaned Data

In [12]:
df.to_parquet('../data_processed/data_jobs_cleaned.parquet', index=False)


### Data Cleaning Summary

- Standardized column names: Stripped whitespace, converted to lowercase, and replaced spaces with underscores for consistency.

- Converted job_posted_date to datetime: Enables proper time-based analysis and trend exploration.

- Dropped mostly empty columns: Removed salary_hour_avg and salary_rate due to excessive missing values and redundancy.

- Retained salary_year_avg: Preserved for potential salary trend analysis — to be filtered or imputed as needed during analysis.

- Dropped 1 row with missing job_title: A core identifying field, so rows without it were removed.

- Filled moderate nulls in categorical columns: Replaced missing values in fields like job_schedule_type, job_location, company_name, etc., with 'Unknown' for completeness.

- Used .loc assignment: Avoided SettingWithCopyWarning by using .loc to safely update DataFrame columns.

- Duplicate Removal – Deferred

While attempting to remove duplicate rows, a TypeError was encountered due to unhashable data types (e.g., Python lists in the job_skills column).
Because these list-based columns are important for downstream analysis, duplicates were not removed at this stage.

This step may be revisited after transforming or narrowing down the relevant columns for comparison.